In [1]:
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama


chat_llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0,
    # other params...
)

In [2]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = chat_llm.bind_tools(tools)

In [3]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

In [4]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={})]

In [5]:


ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'ab6afbf2-87b6-4c53-849e-b832945c99be', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '5d207819-cf5d-40da-a5b7-b591b1945762', 'type': 'tool_call'}]


In [6]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-28T06:56:40.4119913Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1936653500, 'load_duration': 1371535500, 'prompt_eval_count': 231, 'prompt_eval_duration': 175000000, 'eval_count': 44, 'eval_duration': 387000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-0dcd90f5-ffd7-4e32-8bef-ec6601eb82ef-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'ab6afbf2-87b6-4c53-849e-b832945c99be', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '5d207819-cf5d-40da-a5b7-b591b1945762', 'type': 'tool_call'}], usage_metadata={'input_tokens': 231, 'output_tokens': 44, 'total_tokens': 275})]

In [7]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-28T06:56:40.4119913Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1936653500, 'load_duration': 1371535500, 'prompt_eval_count': 231, 'prompt_eval_duration': 175000000, 'eval_count': 44, 'eval_duration': 387000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-0dcd90f5-ffd7-4e32-8bef-ec6601eb82ef-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'ab6afbf2-87b6-4c53-849e-b832945c99be', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '5d207819-cf5d-40da-a5b7-b591b1945762', 'type': 'tool_call'}], usage_metadata={'input_tokens': 231, 'output_tokens': 44, 'total_tokens': 275}),
 ToolMessage(content='36', name='multiply', tool_call_id='ab6afbf2-87b6-4c53-849e-b832945c99be'),


In [8]:
result = llm_with_tools.invoke(messages)

In [9]:
result

AIMessage(content='The result of 3 * 12 is 36.\n\nThe result of 11 + 49 is 60.', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-28T06:58:34.3928149Z', 'done': True, 'done_reason': 'stop', 'total_duration': 668325100, 'load_duration': 22315600, 'prompt_eval_count': 125, 'prompt_eval_duration': 17000000, 'eval_count': 25, 'eval_duration': 224000000, 'message': Message(role='assistant', content='The result of 3 * 12 is 36.\n\nThe result of 11 + 49 is 60.', images=None, tool_calls=None)}, id='run-aae20f5a-b9a9-407d-816f-7e4b251cdd2f-0', usage_metadata={'input_tokens': 125, 'output_tokens': 25, 'total_tokens': 150})

In [10]:
result.content

'The result of 3 * 12 is 36.\n\nThe result of 11 + 49 is 60.'